In [1]:
import pandas as pd
import numpy as np

from horta_exp.introns.fetch_data import get_intron_events

from bokeh.io import push_notebook, output_notebook
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.plotting import figure, show, output_file
from bokeh.charts import BoxPlot, Bar, Histogram
from bokeh.layouts import gridplot
from bokeh.layouts import gridplot

FILE = 'chrom_real_all.pkl'
output_notebook()

Loading BokehJS ...

In [ ]:
introns = get_intron_events()
df = pd.read_pickle(FILE)
df.set_index(['gene', 'intron'], inplace=True)
df.sort_index(inplace=True)

In [ ]:
ii = introns.index.intersection(df.index)
introns = introns.loc[ii,:].sort_index()

In [ ]:
ntests = len(df.index.unique())
print("Number of tests: %d" % ntests)

# Bonferroni within gene-intron
def pval_agg(x):
    return np.clip(np.min(x) * len(x), 0, 1)

def method_agg(x):
    return x[0]
    
def gene_intron_wise(df):
    return df.groupby(level=[0, 1]).agg({
        'lmm-pval': pval_agg,
        'lmm-rank-pval': pval_agg,
        'qep-pval': pval_agg
    })

df = gene_intron_wise(df)

In [ ]:
def method_column(df):
    newdata = pd.DataFrame(columns=('pval', 'method'))

    def append_df(data, df, name):
        pval = df['%s-pval' % name]
        return pd.concat([data, pd.DataFrame({'pval': pval, 'method': name})])
        return data

    newdata = append_df(newdata, df, 'lmm')
    newdata = append_df(newdata, df, 'lmm-rank')
    newdata = append_df(newdata, df, 'qep')
    return newdata

def set_index(data):
    data = method_column(data)
    data.reset_index(inplace=True)
    data['gene'] = data['index'].apply(lambda x: x[0])
    data['intron'] = data['index'].apply(lambda x: int(x[1]))
    del data['index']
    data.set_index(['gene', 'intron'], inplace=True)
    data.sort_index(inplace=True)
    return data

In [ ]:
df = set_index(df)

In [ ]:
group = introns.groupby(level=[0, 1]).agg({
    'ntri': {
        'max': 'max',
        'min': 'min',
        'median': 'median',
        'var': 'var',
        'mean': 'mean',
    }
})
group.sort_index(inplace=True)